In [12]:
import pyproj
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
from rasterstats import zonal_stats
from shapely.geometry import box
import os

import os
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt
import pyproj


# Rutas
RASTER_PATH = "../data/processed/delta_ndbi_2017_2024.tif"
OUT_VECTOR = "../data/vector"
OUT_STATS = "../outputs"
os.makedirs(OUT_VECTOR, exist_ok=True)
os.makedirs(OUT_STATS, exist_ok=True)


# La ruta exacta que encontró tu búsqueda
PROJ_PATH = '/opt/conda/lib/python3.10/site-packages/pyproj/proj_dir/share/proj'

# Configurar variables de entorno para que rasterio y pyproj lo vean
os.environ['PROJ_LIB'] = PROJ_PATH
os.environ['PROJ_DATA'] = PROJ_PATH

# Notificar a la librería pyproj
pyproj.datadir.set_data_dir(PROJ_PATH)

print(f"✅ PROJ configurado exitosamente en: {PROJ_PATH}")

# 1. Crear Grilla (Fishnet) sobre el área del raster
with rasterio.open(RASTER_PATH) as src:
    bounds = src.bounds
    crs = src.crs
    transform = src.transform
    array_ndbi = src.read(1)

# Tamaño de celda aprox 500m (0.0045 grados aprox si es WGS84, o 500 si es UTM)
# Asumiremos WGS84 según tus coordenadas anteriores
step = 0.0045 
xmin, ymin, xmax, ymax = bounds.left, bounds.bottom, bounds.right, bounds.top

grid_cells = []
x_current = xmin
while x_current < xmax:
    y_current = ymin
    while y_current < ymax:
        grid_cells.append(box(x_current, y_current, x_current + step, y_current + step))
        y_current += step
    x_current += step

# Crear GeoDataFrame
grid = gpd.GeoDataFrame({'geometry': grid_cells, 'zone_id': range(len(grid_cells))}, crs=crs)

# 2. Análisis Zonal (Calcular media de cambio NDBI por celda)
stats = zonal_stats(grid, RASTER_PATH, stats="mean count", nodata=np.nan)
df_stats = pd.DataFrame(stats)
df_stats['mean'] = df_stats['mean'].fillna(0) # Rellenar NaN

# Unir estadísticas al GeoDataFrame
grid_final = pd.concat([grid, df_stats], axis=1)
grid_final = grid_final.rename(columns={'mean': 'cambio_ndbi_promedio'})

# 3. Exportar para el Dashboard
# a) CSV para gráficos
grid_final[['zone_id', 'cambio_ndbi_promedio']].to_csv(os.path.join(OUT_STATS, "cambios_por_zona_pudahuel.csv"), index=False)

# b) GeoJSON para el mapa (opcional)
grid_final.to_file(os.path.join(OUT_VECTOR, "pudahuel_grilla.geojson"), driver="GeoJSON")

print(f"✅ Estadísticas zonales generadas: {len(grid_final)} zonas procesadas.")

ModuleNotFoundError: No module named 'pyproj'